In [1095]:
# The necessaries libraries are imported.

import pandas as pd, requests, json, pymysql, re

In [1096]:
# The downloaded data is readed with Pandas and a first sight is gotten.

raw = pd.read_csv(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Finales\Natural Language Processing\src\Datasets\raw.csv")

raw.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


# Observations:

- The prices are in Rupees and they are needed to be changed to dollars.
- The table could have a relational model.

# **`Change to USD`**

In [1097]:
# To change the currency, currencyapi.com will be used to get the actual exchange, 
# so that the data can be up to date at the moment of executing the code.

url = "https://api.currencyapi.com/v3/latest"
headers = {
    'apikey': 'cur_live_XqzUQ4DkY6f0wu0juNSzoFqHkHTVQXZRz4UzpVf3'
}
response = requests.request("GET", url, headers=headers)  # The api is requested.

currencies = json.loads(response.text) # The data is gotten in a string format and then changed to a dictionary.

inr = currencies['data']['INR']['value'] # The dictionary is iterated and the current exchange is gotten.

In [1098]:
# Now, let's change the data to USD; but before that, the columns needs to be cleaned and converted to a numeric type. Now, the data 
# do can be changed to USD using the next formula: USD_Price = INR_Price / INR_OneDollar_Price. And finally the data is rounded two decimals.

raw['discounted_price'] = raw['discounted_price'].apply(lambda n : round(float(n.replace("₹", "").replace(",", ''))/inr, 2))
raw['actual_price'] = raw['actual_price'].apply(lambda n : round(float(n.replace("₹", "").replace(",", ''))/inr, 2))

In [1099]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           1465 non-null   object 
 1   product_name         1465 non-null   object 
 2   category             1465 non-null   object 
 3   discounted_price     1465 non-null   float64
 4   actual_price         1465 non-null   float64
 5   discount_percentage  1465 non-null   object 
 6   rating               1465 non-null   object 
 7   rating_count         1463 non-null   object 
 8   about_product        1465 non-null   object 
 9   user_id              1465 non-null   object 
 10  user_name            1465 non-null   object 
 11  review_id            1465 non-null   object 
 12  review_title         1465 non-null   object 
 13  review_content       1465 non-null   object 
 14  img_link             1465 non-null   object 
 15  product_link         1465 non-null   o

In [1100]:
raw.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,4.77,13.13,64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,2.38,4.17,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,2.38,22.68,90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,3.93,8.35,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,1.84,4.77,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


# **`Relational Model`**

In [1101]:
# First of all, let's see all the features to determine its category.

raw.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')

## Entity Relation

After watching the columns we can notice there are four main categories: products, users and reviews, and the fact table. Finally, we
can arrange them into a SnowFlake model.

![alt text](Images/AmazonSalesERD.png "Title")

In [1102]:
fact = raw[['user_id','product_id', 'actual_price', 'discounted_price', 'discount_percentage', 'review_id']]

product = raw[['product_id', 'product_name','about_product','rating', 'rating_count']]

ProductCategory = raw[['category']]

user = raw[['user_id', 'user_name']]

review = raw[['review_id', 'review_title', 'review_content']]

## Fact Table

In [1103]:
fact.head()

,user_id,product_id,actual_price,discounted_price,discount_percentage,review_id
0,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",B07JW9H4J1,13.13,4.77,64%,"R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K..."
1,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",B098NS6PVG,4.17,2.38,43%,"RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY..."
2,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",B096MSW6CT,22.68,2.38,90%,"R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2..."
3,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",B08HDJ86NZ,8.35,3.93,53%,"R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1..."
4,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",B08CF3B7N1,4.77,1.84,61%,"R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R..."


In [1104]:
fact['user_id'] = fact['user_id'].apply(lambda n : n.split(','))

fact['review_id'] = fact['review_id'].apply(lambda n : n.split(','))

fact.head()

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\2650884391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact['user_id'] = fact['user_id'].apply(lambda n : n.split(','))
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\2650884391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact['review_id'] = fact['review_id'].apply(lambda n : n.split(','))


,user_id,product_id,actual_price,discounted_price,discount_percentage,review_id
0,"[AG3D6O4STAQKAY2UVGEUV46KN35Q, AHMY5CWJMMK5BJR...",B07JW9H4J1,13.13,4.77,64%,"[R3HXWT0LRP0NMF, R2AJM3LFTLZHFO, R6AQJGUP6P86,..."
1,"[AECPFYFQVRUWC3KGNLJIOREFP5LQ, AGYYVPDD7YG7FYN...",B098NS6PVG,4.17,2.38,43%,"[RGIQEG07R9HS2, R1SMWZQ86XIN8U, R2J3Y1WL29GWDE..."
2,"[AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA, AESFLDV2PT363T2...",B096MSW6CT,22.68,2.38,90%,"[R3J3EQQ9TZI5ZJ, R3E7WBGK7ID0KV, RWU79XKQ6I1QF..."
3,"[AEWAZDZZJLQUYVOVGBEUKSLXHQ5A, AG5HTSFRRE6NL3M...",B08HDJ86NZ,8.35,3.93,53%,"[R3EEUZKKK9J36I, R3HJVYCLYOY554, REDECAZ7AMPQC..."
4,"[AE3Q6KSUK5P75D5HFYHCRAOLODSA, AFUGIFH5ZAFXRDS...",B08CF3B7N1,4.77,1.84,61%,"[R1BP4L2HH9TFUP, R16PVJEXKV6QZS, R2UPDB81N66T4..."


In [1105]:
fact = fact.explode(['user_id','review_id'], ignore_index=True)

fact.dropna(inplace=True)

fact.head()

,user_id,product_id,actual_price,discounted_price,discount_percentage,review_id
0,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,13.13,4.77,64%,R3HXWT0LRP0NMF
1,AHMY5CWJMMK5BJRBBSNLYT3ONILA,B07JW9H4J1,13.13,4.77,64%,R2AJM3LFTLZHFO
2,AHCTC6ULH4XB6YHDY6PCH2R772LQ,B07JW9H4J1,13.13,4.77,64%,R6AQJGUP6P86
3,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,B07JW9H4J1,13.13,4.77,64%,R1KD19VHEDV0OR
4,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,B07JW9H4J1,13.13,4.77,64%,R3C02RMYQMK6FC


## Product Table

In [1106]:
product.head()

,product_id,product_name,about_product,rating,rating_count
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,4.2,"24,269"
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",4.0,"43,994"
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,3.9,"7,928"
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,4.2,"94,363"
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,4.2,"16,905"


In [1107]:
product[product['product_id'].duplicated() == True]

,product_id,product_name,about_product,rating,rating_count
369,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,4.2,"24,270"
377,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",4.0,"43,993"
379,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,3.9,"7,928"
392,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,4.2,"94,364"
393,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,4.2,"16,905"
...,...,...,...,...,...
1007,B0B9BXKBC7,"WeCool S5 Long Selfie Stick, with Large Reinfo...","64"" Tall Cell Phone Tripods with 7 section adj...",4.6,245
1010,B07GVGTSLN,Wayona Usb Type C Fast Charger Cable Fast Char...,Fast Charge & Data Sync: Fast charge& data tra...,4.2,"10,576"
1017,B0B9BD2YL4,KINGONE Wireless Charging Pencil (2nd Generati...,【2022 latest iPad Magnetic Charging】 The lates...,4.5,"1,526"
1018,B0BMXMLSMM,Lapster 65W compatible for OnePlus Dash Warp C...,-1 meter type c to c cable fast charging cable...,4.5,127


In [1108]:
product = product.drop_duplicates(subset=['product_id'])

product.dropna(inplace=True)

product

,product_id,product_name,about_product,rating,rating_count
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,4.2,"24,269"
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",4.0,"43,994"
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,3.9,"7,928"
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,4.2,"94,363"
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,4.2,"16,905"
...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,4,"1,090"
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,"230 Volts, 400 watts, 1 Year",4.1,"4,118"
1462,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,International design and styling|Two heat sett...,3.6,468
1463,B00J5DYCCA,Havells Ventil Air DSP 230mm Exhaust Fan (Pist...,Fan sweep area: 230 MM ; Noise level: (40 - 45...,4,"8,031"


In [1109]:
product['rating_count'] = product['rating_count'].apply(lambda n : int(n.replace(',', '')))

product.head()

,product_id,product_name,about_product,rating,rating_count
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,4.2,24269
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",4.0,43994
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,3.9,7928
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,4.2,94363
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,4.2,16905


In [1110]:
product['rating'] = product['rating'].apply(lambda n : float(n) if bool(re.match(r"^-?\d+(\.\d+)?$", n)) else 0.0)

product.head()

,product_id,product_name,about_product,rating,rating_count
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,4.2,24269
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",4.0,43994
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,3.9,7928
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,4.2,94363
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,4.2,16905


In [1111]:
product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1349 entries, 0 to 1464
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     1349 non-null   object 
 1   product_name   1349 non-null   object 
 2   about_product  1349 non-null   object 
 3   rating         1349 non-null   float64
 4   rating_count   1349 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 63.2+ KB


## Product Category Table

In [1112]:
ProductCategory

,category
0,Computers&Accessories|Accessories&Peripherals|...
1,Computers&Accessories|Accessories&Peripherals|...
2,Computers&Accessories|Accessories&Peripherals|...
3,Computers&Accessories|Accessories&Peripherals|...
4,Computers&Accessories|Accessories&Peripherals|...
...,...
1460,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...
1461,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...
1462,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe..."
1463,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|E..."


In [1113]:
ProductCategory[ProductCategory.duplicated() == True]

,category
1,Computers&Accessories|Accessories&Peripherals|...
2,Computers&Accessories|Accessories&Peripherals|...
3,Computers&Accessories|Accessories&Peripherals|...
4,Computers&Accessories|Accessories&Peripherals|...
5,Computers&Accessories|Accessories&Peripherals|...
...,...
1460,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...
1461,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...
1462,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe..."
1463,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|E..."


In [1114]:
ProductCategory = ProductCategory.drop_duplicates(subset=['category'])


ProductCategory

,category
0,Computers&Accessories|Accessories&Peripherals|...
8,Computers&Accessories|NetworkingDevices|Networ...
12,"Electronics|HomeTheater,TV&Video|Accessories|C..."
16,"Electronics|HomeTheater,TV&Video|Televisions|S..."
48,"Electronics|HomeTheater,TV&Video|Accessories|R..."
...,...
1418,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...
1422,"Home&Kitchen|Heating,Cooling&AirQuality|Parts&..."
1432,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...
1439,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|P..."


In [1115]:
ProductCategory['category'] = ProductCategory['category'].apply(lambda n : n.split('|'))

ProductCategory = ProductCategory.explode('category', ignore_index=True)

ProductCategory = ProductCategory.drop_duplicates()

ProductCategory.dropna(inplace=True)

ProductCategory

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3844170174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ProductCategory['category'] = ProductCategory['category'].apply(lambda n : n.split('|'))


,category
0,Computers&Accessories
1,Accessories&Peripherals
2,Cables&Accessories
3,Cables
4,USBCables
...,...
885,StandMixers
889,PedestalFans
894,VacuumAccessories
895,VacuumBags


## User Table

In [1116]:
user.drop_duplicates(inplace=True)

user['user_id'] = user['user_id'].apply(lambda n : n.split(","))

user['user_name'] = user['user_name'].apply(lambda n : n.split(","))

user

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\1024341032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user.drop_duplicates(inplace=True)
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\1024341032.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['user_id'] = user['user_id'].apply(lambda n : n.split(","))
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\1024341032.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,user_id,user_name
0,"[AG3D6O4STAQKAY2UVGEUV46KN35Q, AHMY5CWJMMK5BJR...","[Manav, Adarsh gupta, Sundeep, S.Sayeed Ahmed,..."
1,"[AECPFYFQVRUWC3KGNLJIOREFP5LQ, AGYYVPDD7YG7FYN...","[ArdKn, Nirbhay kumar, Sagar Viswanathan, Asp,..."
2,"[AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA, AESFLDV2PT363T2...","[Kunal, Himanshu, viswanath, sai niharka, saqi..."
3,"[AEWAZDZZJLQUYVOVGBEUKSLXHQ5A, AG5HTSFRRE6NL3M...","[Omkar dhale, JD, HEMALATHA, Ajwadh a., amar s..."
4,"[AE3Q6KSUK5P75D5HFYHCRAOLODSA, AFUGIFH5ZAFXRDS...","[rahuls6099, Swasat Borah, Ajay Wadke, Pranali..."
...,...,...
1460,"[AHITFY6AHALOFOHOZEOC6XBP4FEA, AFRABBODZJZQB6Z...","[Prabha ds, Raghuram bk, Real Deal, Amazon Cus..."
1461,"[AFG5FM3NEMOL6BNFRV2NK5FNJCHQ, AGEINTRN6Z563RM...","[Manu Bhai, Naveenpittu, Evatira Sangma, JAGAN..."
1462,"[AGVPWCMAHYQWJOQKMUJN4DW3KM5Q, AF4Q3E66MY4SR7Y...","[Nehal Desai, Danish Parwez, Amazon Customer, ..."
1463,"[AF2JQCLSCY3QJATWUNNHUSVUPNQQ, AFDMLUXC5LS5RXD...","[Shubham Dubey, E.GURUBARAN, Mayank S., eusuf ..."


In [1117]:
user['user_id_count'] = user['user_id'].apply(lambda n : len(n))

user['user_name_count'] = user['user_name'].apply(lambda n : len(n))

user

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3991757026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['user_id_count'] = user['user_id'].apply(lambda n : len(n))
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3991757026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['user_name_count'] = user['user_name'].apply(lambda n : len(n))


,user_id,user_name,user_id_count,user_name_count
0,"[AG3D6O4STAQKAY2UVGEUV46KN35Q, AHMY5CWJMMK5BJR...","[Manav, Adarsh gupta, Sundeep, S.Sayeed Ahmed,...",8,8
1,"[AECPFYFQVRUWC3KGNLJIOREFP5LQ, AGYYVPDD7YG7FYN...","[ArdKn, Nirbhay kumar, Sagar Viswanathan, Asp,...",8,8
2,"[AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA, AESFLDV2PT363T2...","[Kunal, Himanshu, viswanath, sai niharka, saqi...",8,8
3,"[AEWAZDZZJLQUYVOVGBEUKSLXHQ5A, AG5HTSFRRE6NL3M...","[Omkar dhale, JD, HEMALATHA, Ajwadh a., amar s...",8,8
4,"[AE3Q6KSUK5P75D5HFYHCRAOLODSA, AFUGIFH5ZAFXRDS...","[rahuls6099, Swasat Borah, Ajay Wadke, Pranali...",8,8
...,...,...,...,...
1460,"[AHITFY6AHALOFOHOZEOC6XBP4FEA, AFRABBODZJZQB6Z...","[Prabha ds, Raghuram bk, Real Deal, Amazon Cus...",8,8
1461,"[AFG5FM3NEMOL6BNFRV2NK5FNJCHQ, AGEINTRN6Z563RM...","[Manu Bhai, Naveenpittu, Evatira Sangma, JAGAN...",8,8
1462,"[AGVPWCMAHYQWJOQKMUJN4DW3KM5Q, AF4Q3E66MY4SR7Y...","[Nehal Desai, Danish Parwez, Amazon Customer, ...",8,8
1463,"[AF2JQCLSCY3QJATWUNNHUSVUPNQQ, AFDMLUXC5LS5RXD...","[Shubham Dubey, E.GURUBARAN, Mayank S., eusuf ...",8,8


In [1118]:
user = user[~(user['user_id_count'] != user['user_name_count'])][['user_id', 'user_name']]

user.head()

,user_id,user_name
0,"[AG3D6O4STAQKAY2UVGEUV46KN35Q, AHMY5CWJMMK5BJR...","[Manav, Adarsh gupta, Sundeep, S.Sayeed Ahmed,..."
1,"[AECPFYFQVRUWC3KGNLJIOREFP5LQ, AGYYVPDD7YG7FYN...","[ArdKn, Nirbhay kumar, Sagar Viswanathan, Asp,..."
2,"[AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA, AESFLDV2PT363T2...","[Kunal, Himanshu, viswanath, sai niharka, saqi..."
3,"[AEWAZDZZJLQUYVOVGBEUKSLXHQ5A, AG5HTSFRRE6NL3M...","[Omkar dhale, JD, HEMALATHA, Ajwadh a., amar s..."
4,"[AE3Q6KSUK5P75D5HFYHCRAOLODSA, AFUGIFH5ZAFXRDS...","[rahuls6099, Swasat Borah, Ajay Wadke, Pranali..."


In [1119]:
user = user.explode(column=['user_id','user_name'], ignore_index=True).drop_duplicates()

user.dropna(inplace=True)

user.head()

,user_id,user_name
0,AG3D6O4STAQKAY2UVGEUV46KN35Q,Manav
1,AHMY5CWJMMK5BJRBBSNLYT3ONILA,Adarsh gupta
2,AHCTC6ULH4XB6YHDY6PCH2R772LQ,Sundeep
3,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,S.Sayeed Ahmed
4,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,jaspreet singh


## Review Table

In [1120]:
review['review_id'] = review['review_id'].apply(lambda n : n.split(','))

review['review_title'] = review['review_title'].apply(lambda n : n.split(','))

review['review_content'] = review['review_content'].apply(lambda n : n.split(','))

review['review_id_count'] = review['review_id'].apply(lambda n : len(n))

review['review_title_count'] = review['review_title'].apply(lambda n : len(n))

review['review_content_count'] = review['review_content'].apply(lambda n : len(n))

review

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3456486939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['review_id'] = review['review_id'].apply(lambda n : n.split(','))
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3456486939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['review_title'] = review['review_title'].apply(lambda n : n.split(','))
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3456486939.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,review_id,review_title,review_content,review_id_count,review_title_count,review_content_count
0,"[R3HXWT0LRP0NMF, R2AJM3LFTLZHFO, R6AQJGUP6P86,...","[Satisfied, Charging is really fast, Value for...",[Looks durable Charging is fine tooNo complain...,8,8,10
1,"[RGIQEG07R9HS2, R1SMWZQ86XIN8U, R2J3Y1WL29GWDE...","[A Good Braided Cable for Your Type C Device, ...",[I ordered this cable to connect my phone to A...,8,8,15
2,"[R3J3EQQ9TZI5ZJ, R3E7WBGK7ID0KV, RWU79XKQ6I1QF...","[Good speed for earlier versions, Good Product...","[Not quite durable and sturdy, https://m.media...",8,8,8
3,"[R3EEUZKKK9J36I, R3HJVYCLYOY554, REDECAZ7AMPQC...","[Good product, Good one, Nice, Really nice pro...","[Good product, long wire, Charges good, Nice, ...",8,8,10
4,"[R1BP4L2HH9TFUP, R16PVJEXKV6QZS, R2UPDB81N66T4...","[As good as original, Decent, Good one for sec...","[Bought this instead of original apple, does ...",8,8,20
...,...,...,...,...,...,...
1460,"[R3G3XFHPBFF0E8, R3C0BZCD32EIGW, R2EBVBCN9QPD9...","[Received the product without spanner, Excelle...","[I received product without spanner, Excellent...",8,8,8
1461,"[R3DDL2UPKQ2CK9, R2SYYU1OATVIU5, R1VM993161IYR...","[ok, everything was good couldn't return bcoz ...","[ok, got everything as mentioned but the measu...",8,8,8
1462,"[R1TLRJVW4STY5I, R2O455KRN493R1, R3Q5MVGBRIAS2...","[very good, Work but front melt after 2 month,...","[plastic but cool body , u have to find sturdy...",8,8,18
1463,"[R39Q2Y79MM9SWK, R3079BG1NIH6MB, R29A31ZELTZNJ...","[Fan Speed is slow, Good quality, Good product...",[I have installed this in my kitchen working f...,8,8,8


In [1121]:
review = review[review['review_title_count'] == review['review_content_count']][review['review_id_count'] == review['review_title_count']].explode(['review_id', 'review_title','review_content'], ignore_index=True)[['review_id', 'review_title','review_content']]

review.dropna(inplace=True)

review.drop_duplicates(subset=['review_id'], inplace=True)

review.head()

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\3195243152.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  review = review[review['review_title_count'] == review['review_content_count']][review['review_id_count'] == review['review_title_count']].explode(['review_id', 'review_title','review_content'], ignore_index=True)[['review_id', 'review_title','review_content']]


,review_id,review_title,review_content
0,R3J3EQQ9TZI5ZJ,Good speed for earlier versions,Not quite durable and sturdy
1,R3E7WBGK7ID0KV,Good Product,https://m.media-amazon.com/images/W/WEBP_40237...
2,RWU79XKQ6I1QF,Working good,Working good
3,R25X4TBMPY91LX,Good for the price,https://m.media-amazon.com/images/W/WEBP_40237...
4,R27OK7G99VK0TR,Good,Product


## MySQL Connection.

In [1122]:
# Let's create the database.

with pymysql.connect(user='root', passwd='ningunaok12OK@') as connection:
    with connection.cursor() as cursor:
        cursor.execute("DROP DATABASE IF EXISTS AmazonSales;")
        cursor.execute("CREATE DATABASE IF NOT EXISTS AmazonSales;")

In [1123]:
# Let's create the tables with their respective primary and foreign keys.

with pymysql.connect(user='root', passwd='ningunaok12OK@', database='AmazonSales') as connection:
    with connection.cursor() as cursor:
        cursor.execute("""
                       CREATE TABLE IF NOT EXISTS User(
                            UserID VARCHAR(200) PRIMARY KEY,
                            UserName VARCHAR(255)
                       );
                       """)
        cursor.execute("""
                    CREATE TABLE IF NOT EXISTS ProductCategory(
                       CategoryID INT PRIMARY KEY AUTO_INCREMENT,
                       Category VARCHAR(255)
                       ); 
                       """)
        cursor.execute("""
                    CREATE TABLE IF NOT EXISTS Product(
                       ProductID VARCHAR(100) PRIMARY KEY,
                       ProductName VARCHAR(1000),
                       AboutProduct VARCHAR(5000),
                       Rating FLOAT,
                       RatingCount INTEGER,
                       CategoryID INT,
                       FOREIGN KEY (CategoryID) REFERENCES ProductCategory(CategoryID)
                       );
                        """)
        cursor.execute("""
                    CREATE TABLE IF NOT EXISTS Review (
                       ReviewID VARCHAR(255) PRIMARY KEY,
                       ReviewTitle VARCHAR(255),
                       ReviewContent VARCHAR(1000)
                       );
                        """)
        cursor.execute("""
                    CREATE TABLE IF NOT EXISTS Fact (
                       SalesID INT PRIMARY KEY AUTO_INCREMENT,
                       UserID VARCHAR(200),
                       ProductID VARCHAR(100),
                       Price FLOAT,
                       DiscountPercentage VARCHAR(4),
                       PriceDiscounted FLOAT,
                       ReviewID VARCHAR(255)
                       );
                        """)

In [1124]:
# Let's insert the data in the respective table.

with pymysql.connect(user='root', passwd='ningunaok12OK@', database='AmazonSales') as connection:
    with connection.cursor() as cursor:
        cursor.executemany("""
                        INSERT INTO User (UserID, UserName) VALUES (%s, %s);
                            """, [tuple(i) for i in user.to_numpy()])
        cursor.executemany("""
                        INSERT INTO Review (ReviewID, ReviewTitle, ReviewContent) VALUES (%s, %s, %s);
                            """, [tuple(i) for i in review.to_numpy()])
        cursor.executemany("""
                        INSERT INTO Product (ProductID, ProductName, AboutProduct, Rating, RatingCount) VALUES (%s, %s, %s, %s, %s);
                            """, [tuple(i) for i in product.to_numpy()])
        cursor.executemany("""
                        INSERT INTO ProductCategory (Category) VALUES (%s);
                            """, [tuple(i) for i in ProductCategory.to_numpy()])
        cursor.executemany("""
                        INSERT INTO Fact (UserID, ProductID, Price, PriceDiscounted, DiscountPercentage, ReviewID) VALUES (%s, %s, %s, %s, %s, %s);
                            """, [tuple(i) for i in fact.to_numpy()])
    connection.commit()

In [1125]:
# Let's full the CategoryID column from the Product table.

with pymysql.connect(user='root', passwd='ningunaok12OK@', database='AmazonSales') as connection:
    category_id = pd.read_sql("""
                              SELECT *
                              FROM ProductCategory;
                              """, connection)

category = raw[['product_id', 'category']]

category['category'] = category['category'].apply(lambda n : n.split('|'))

category = category.explode('category', ignore_index=True).drop_duplicates('product_id').dropna()

category_id_2 = pd.merge(category, category_id, left_on='category', right_on='Category', how='left')

category_id_3 = product.merge(category_id_2, on='product_id', how='left')[['product_id', 'CategoryID']]

with pymysql.connect(user='root', passwd='ningunaok12OK@', database='AmazonSales') as connection:
    with connection.cursor() as cursor:
        for i, j in category_id_3.to_numpy():
            cursor.execute("UPDATE Product SET CategoryID = %s WHERE ProductID = %s;", (j, i))
    connection.commit()

C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\179696720.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  category_id = pd.read_sql("""
C:\Users\roylo\AppData\Local\Temp\ipykernel_88628\179696720.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category['category'] = category['category'].apply(lambda n : n.split('|'))
